In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.4'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.4.0-dev20201007


In [2]:
#loss function
def nll(scores_pos, scores_neg):
    neg_exp = tf.exp(scores_neg)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.math.log(softmax_score)
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adam(learning_rate=0.01)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss, NLLMulticlass
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5, 'reduction': 'sum'})
loss = NLLMulticlass({'reduction': 'sum'})
model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300,
                                     scoring_type='ComplEx')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

start = time.time()
model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])

end = time.time()
print('Time taken:', end-start)

Epoch 1/10
     28/Unknown - 2s 67ms/step - loss: 1.6903
Epoch 00001: loss improved from inf to 1.68641, saving model to ./chkpt1
29/29 [==============================] - 2s 68ms/step - loss: 1.6864
Epoch 2/10
29/29 [==============================] - ETA: 0s - loss: 1.0768
Epoch 00002: loss improved from 1.68641 to 1.07681, saving model to ./chkpt1
29/29 [==============================] - 1s 23ms/step - loss: 1.0768
Epoch 3/10
29/29 [==============================] - ETA: 0s - loss: 0.7676
Epoch 00003: loss improved from 1.07681 to 0.76761, saving model to ./chkpt1
29/29 [==============================] - 1s 23ms/step - loss: 0.7676
Epoch 4/10
29/29 [==============================] - ETA: 0s - loss: 0.6026
Epoch 00004: loss improved from 0.76761 to 0.60263, saving model to ./chkpt1
29/29 [==============================] - 1s 23ms/step - loss: 0.6026
Epoch 5/10
26/29 [=========================>....] - ETA: 0s - loss: 0.5078
9 triples containing invalid keys skipped!
177/177 [===========

In [4]:
pred = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(np.sort(pred))


28 triples containing invalid keys skipped!
(20438,)
[-2.6117415 -2.5582657 -2.4347568 ... 30.218767  31.553259  41.35157  ]


In [5]:
model.calibrate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                batch_size=10000, positive_base_rate=0.5, epochs=20)


28 triples containing invalid keys skipped!


In [6]:
out = model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
print(np.sort(out))
print(np.argsort(out))
pred_out = model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')
print(np.sort(pred_out))
print(np.argsort(pred_out))



28 triples containing invalid keys skipped!
[0.44457036 0.44533417 0.44709924 ... 0.8423783  0.8523569  0.9105091 ]
[ 4066   611 18634 ...   990 10437 14612]

28 triples containing invalid keys skipped!
[-2.6117415 -2.5582657 -2.4347568 ... 30.218767  31.553259  41.35157  ]
[ 4066   611 18634 ...   990 10437 14612]


In [7]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 52s 254ms/step
Time taken: 52.35202598571777


(311.1254525883159,
 0.3877527518637535,
 0.3215334181426754,
 0.5160485370388492,
 20438)

In [8]:
model.save_weights('./calibrated_model')

## Load Checkpoint and evaluate

In [9]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='ComplEx')
loaded_model.load_weights('./calibrated_model')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 52s 254ms/step
Time taken: 52.437538146972656


(311.1254525883159,
 0.3877527518637535,
 0.3215334181426754,
 0.5160485370388492,
 20438)

In [10]:
pred = loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100)
print(pred.shape)
print(pred)


28 triples containing invalid keys skipped!
(20438,)
[14.611031   9.369465   7.550661  ...  5.55168    0.9343749  5.470505 ]


In [12]:
out = loaded_model.predict_proba('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', batch_size=10000)
np.argsort(out), np.argsort(loaded_model.predict('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt')), np.sort(out)


28 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!


(array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([ 4066,   611, 18634, ...,   990, 10437, 14612]),
 array([0.44457036, 0.44533417, 0.44709924, ..., 0.8423783 , 0.8523569 ,
        0.9105091 ], dtype=float32))

# Training/eval with partition 

## Training with RandomEdges partitioner

In [7]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                     use_filter=False,
                                    use_indexer=True)

In [8]:


# Choose the partitioner 
partitioner = PARTITION_ALGO_REGISTRY.get('RandomEdges')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 12.886MB, consumed: 12.886MB; exec time: 31.461s


In [9]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)

In [10]:

start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=2)
print((time.time()-start))

Epoch 2/2
274/274 [==============================] - 16s 58ms/step - loss: 1.5947
28.386675596237183


In [11]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=partitioned_model.data_handler.get_mapper())


28 triples containing invalid keys skipped!


In [12]:

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 129s 2s/step
Time taken: 129.33987951278687


(2613.1418925530875, 0.07781069124506454, 0.0, 0.22360309227908798, 20438)

## Save and Load

In [13]:
partitioned_model.save_weights('./best_model')

In [14]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./best_model')

In [15]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=DummyBackend,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        use_indexer=loaded_part_model.data_indexer)


28 triples containing invalid keys skipped!


In [16]:


start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)

53/53 [==============================] - 131s 2s/step
Time taken: 131.47749257087708


(2613.1418925530875, 0.07781069124506454, 0.0, 0.22360309227908798, 20438)

# Training/eval with partition (default Partitioning Approach)

In [17]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY


In [18]:
optim = tf.optimizers.Adam(learning_rate=0.001, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')
partitioned_model.compile(optimizer=optim, loss=nll)


start = time.time()
partitioned_model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                     batch_size=1000, use_partitioning=True,
                     epochs=10)
print((time.time()-start))

_split: memory before: 102.66MB, after: 115.36MB, consumed: 12.705MB; exec time: 86.93s
Epoch 2/10
277/277 [==============================] - 9s 34ms/step - loss: 1.7309
Epoch 3/10
277/277 [==============================] - 9s 32ms/step - loss: 1.6420
Epoch 4/10
277/277 [==============================] - 9s 31ms/step - loss: 1.5553
Epoch 5/10
277/277 [==============================] - 9s 32ms/step - loss: 1.4762
Epoch 6/10
277/277 [==============================] - 9s 31ms/step - loss: 1.4055
Epoch 7/10
277/277 [==============================] - 9s 31ms/step - loss: 1.3427
Epoch 8/10
277/277 [==============================] - 9s 32ms/step - loss: 1.2868
Epoch 9/10
277/277 [==============================] - 9s 32ms/step - loss: 1.2369
Epoch 10/10
277/277 [==============================] - 9s 32ms/step - loss: 1.1923
175.27707862854004


In [19]:


start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!
53/53 [==============================] - 132s 2s/step
Time taken: 131.93697667121887


(789.7641892553088, 0.08769968002895012, 0.0, 0.23776788335453566, 20438)

## Filtered evaluation

In [20]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=400,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10), len(ranks)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
53/53 [==============================] - 297s 6s/step
Time taken: 297.1006762981415


(664.6330365006361,
 0.1936046545792393,
 0.13240043057050593,
 0.3117477248263039,
 20438)